<a href="https://colab.research.google.com/github/THS-ST/ocr-models/blob/main/OCR_Models_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **OCR Benchmark Notebook**

This serves to **compare multiple OCR libraries** on your own images. It includes:
- A sample run for the following models: Tesseract (via `pytesseract`), EasyOCR, PaddleOCR, DocTR, TrOCR, and Keras-OCR
- A simple evaluation using **WER/CER (via `jiwer`)** *(TBA)*
- Runtime timing per image + per model, and a summary chart *(TBA)*

> **Folder layout:**
>
> - `data/images/` → test images (PNG/JPG/TIFF/PDF-as-images)
> - `data/ground_truth.csv` → optional ground-truth file with columns: `filename,text`  
>   (filenames should match exactly the images in `data/images/`)
>

**Acknowledgement**: Thank you to Ang et al. for allowing us to use their synthetic typewritten prescription dataset for our study.


# **Installations**

In [1]:
!git clone https://github.com/THS-ST/ocr-models

fatal: destination path 'ocr-models' already exists and is not an empty directory.


In [2]:

# --- Install Python packages (run once per environment) ---
%pip -q install paddlepaddle
%pip -q install pytesseract easyocr paddleocr doctr[torch] jiwer opencv-python pillow matplotlib rapidfuzz
%pip -q install python-doctr

# Extra (uncomment as needed):
%pip -q install keras-ocr  # requires TensorFlow 2.x and may be heavy
%pip -q install transformers timm accelerate # for TrOCR (Hugging Face)
%pip -q install mmocr==1.0.1 mmengine==0.10.4 mmdet==3.2.0  # complex; only if you want OpenMMLab


**Tesseract binary is required** for `pytesseract`.

In [3]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# **Initial Imports and Setup**


## System Notes
- GPU is optional but speeds up deep learning models (PaddleOCR, DocTR, Keras-OCR, TrOCR).
- If you see CUDA errors, switch to CPU by installing CPU-only wheels or setting the appropriate flags.


In [4]:

import os, time, json, string, glob, io, math
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Callable, List, Tuple

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

# Evaluation
import jiwer

DATA_DIR = Path("ocr-models/data")
IMAGES_DIR = DATA_DIR / "images"
GT_CSV = DATA_DIR / "ground_truth.csv"

IMAGES_DIR.mkdir(parents=True, exist_ok=True)
print("Expecting images in:", IMAGES_DIR.resolve())
print("Optional ground-truth CSV at:", GT_CSV.resolve())


Expecting images in: /content/ocr-models/data/images
Optional ground-truth CSV at: /content/ocr-models/data/ground_truth.csv


We will load our images with OpenCV (BGR) and return an RGB np.ndarray.

In [5]:
def load_image(path: Path):
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

The following are additional helper functions for loading images and ground truth values.

In [6]:
def list_images(d: Path) -> List[Path]:
    exts = ["*.png", "*.jpg", "*.JPG", "*.jpeg", "*.tif", "*.tiff", "*.bmp"]
    files = []
    for e in exts:
        files.extend(sorted(d.glob(e)))
    return files

def load_ground_truth(gt_csv: Path) -> pd.DataFrame:
    if gt_csv.exists():
        df = pd.read_csv(gt_csv)
        assert {"filename", "text"}.issubset(df.columns), "ground_truth.csv must have columns: filename,text"
        return df
    else:
        return pd.DataFrame(columns=["filename", "text"])

def get_gt_for(fname: str, gt_df: pd.DataFrame) -> str:
    row = gt_df.loc[gt_df["filename"] == fname]
    return "" if row.empty else str(row.iloc[0]["text"])

Next, we define robust text normalization for fair evaluation.

In [7]:
transform = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.Strip(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
])

def compute_metrics(pred: str, ref: str) -> dict:
    if not ref:
        return {"wer": math.nan, "cer": math.nan}
    wer = jiwer.wer(ref, pred, truth_transform=transform, hypothesis_transform=transform)
    cer = jiwer.cer(ref, pred, truth_transform=transform, hypothesis_transform=transform)
    return {"wer": wer, "cer": cer}


Now we can call the functions to load the images and ground truth values.

In [8]:
images = list_images(IMAGES_DIR)
# No ground truth values for now, uncomment as needed
# gt_df = load_ground_truth(GT_CSV)

In [9]:
images

[PosixPath('ocr-models/data/images/1.jpg'),
 PosixPath('ocr-models/data/images/10.jpg'),
 PosixPath('ocr-models/data/images/2.jpg'),
 PosixPath('ocr-models/data/images/3.jpg'),
 PosixPath('ocr-models/data/images/4.jpg'),
 PosixPath('ocr-models/data/images/5.jpg'),
 PosixPath('ocr-models/data/images/6.jpg'),
 PosixPath('ocr-models/data/images/7.jpg'),
 PosixPath('ocr-models/data/images/8.jpg'),
 PosixPath('ocr-models/data/images/9.jpg')]

We will define the following dataclass for our OCR results.

In [10]:
from dataclasses import dataclass

@dataclass
class OCRResult:
    filename: str
    model: str
    pred_text: str
    secs: float

# **1.0: Tesseract**
One of the most widely used OCR engines globally is Tesseract, developed by Google, it utilizes a long short-term memory (LSTM) based neural network for text recognition and has support for over 100 languages (Smith, 2007). It excels at recognizing printed text from high quality images, but performance significantly drops with degraded images or complex layouts. The model also faces challenges with handwriting recognition and images with significant distortions such as noise or skew (Gupta et al., 2021).

We will use the ``pytesseract`` module.

In [11]:
import pytesseract

Now we will define our tesseractOCR function. This will take an ``np.ndarray`` and return an extracted text string.

In [12]:
def ocr_tesseract(img_rgb: np.ndarray) -> str:
    pil_img = Image.fromarray(img_rgb)
    text = pytesseract.image_to_string(pil_img)
    return text.strip()

The results will be stored in this array.

In [13]:
tesseract_results = []

Next, we will call this function for all the images in the directory.

In [14]:
current_model = "tesseract"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_tesseract(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        tesseract_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/1.jpg...
Processing: ocr-models/data/images/10.jpg...
Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...


Let's turn the array into a dataframe for a better look at our OCR results.

In [15]:
df = pd.DataFrame([r.__dict__ for r in tesseract_results])
df

,filename,model,pred_text,secs
0,1.jpg,tesseract,1. Fluimucil 600mg\n#15\n\nSig: Dissolve 1 tab...,2.355070
1,10.jpg,tesseract,Amoxicillin 500mg cap #21\nSig: take 1 cap 3x ...,1.414121
2,2.jpg,tesseract,Fluimucil 600mg #15\nDissoWve 1 tablet in % gl...,1.207397
3,3.jpg,tesseract,Fluimucil 600mg #15\nDissolve 1 tablet in % gl...,1.438213
4,4.jpg,tesseract,1. DecolgeniNeozep\n#15\nSig: Take 1 tablet 3x...,1.264141
5,5.jpg,tesseract,Decolgen/Neozep #15\n1 tablet 3x/day for 5 day...,0.668648
6,6.jpg,tesseract,Decolgen/Neozep #15\n1 tablet three times a da...,1.123239
7,7.jpg,tesseract,1. Betadine Gargle\na\nSig: Gargle 3x/day\n\n2...,0.668190
8,8.jpg,tesseract,Betadine Gargle a\nGargle 3x/day\n\nDiffiam Lo...,0.705207
9,9.jpg,tesseract,Betadine Gargle a\nGargle three times a day\n\...,1.302928


Since newline characters are printed as is in a dataframe, let's try isolating the text for a better visualization.

In [16]:
for r in tesseract_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[1.jpg]
1. Fluimucil 600mg
#15

Sig: Dissolve 1 tablet in % glass water once a
day for 5 days

2. Immunosin 500mg
#21

Sig: Take 1 tablet 3x/day for 7 days


[10.jpg]
Amoxicillin 500mg cap #21
Sig: take 1 cap 3x a day x 1 week


[2.jpg]
Fluimucil 600mg #15
DissoWve 1 tablet in % glass water 1x/day.

Immunosin 500mg #21
axiday


[3.jpg]
Fluimucil 600mg #15
Dissolve 1 tablet in % glass water once a day.

Immunosin 500mg #21
Three times a day


[4.jpg]
1. DecolgeniNeozep
#15
Sig: Take 1 tablet 3x/day for 5 days

2. Loratadine 10mg
#21

Sig: Take 1 tablet at breakfast for 14 days


[5.jpg]
Decolgen/Neozep #15
1 tablet 3x/day for 5 days

Loratadine 10mg #21
1 tablet 1x/day at breakfast for 14 days


[6.jpg]
Decolgen/Neozep #15
1 tablet three times a day for 5 days

Loratadine 10mg #21
1 tablet once a day at breakfast for 14 days


[7.jpg]
1. Betadine Gargle
a
Sig: Gargle 3x/day

2. Difflam lozenges
#15
Sig: Every 4 hours


[8.jpg]
Betadine Gargle a
Gargle 3x/day

Diffiam Lozenges #15
1x/4ho

# **2.0: EasyOCR**


In [17]:
import easyocr

In [18]:
easyocr_reader = easyocr.Reader(['en'], gpu=True if os.environ.get("USE_CUDA", "0") == "1" else False)

In [19]:
def ocr_easyocr(img_rgb: np.ndarray) -> str:
    result = easyocr_reader.readtext(img_rgb, detail=0, paragraph=True)
    return "\n".join([r.strip() for r in result if r.strip()])

In [20]:
easyocr_results = []

In [21]:
current_model = "easyocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_easyocr(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        easyocr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/1.jpg...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Processing: ocr-models/data/images/10.jpg...
Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...


In [22]:
eocr_df = pd.DataFrame([r.__dict__ for r in easyocr_results])
eocr_df

,filename,model,pred_text,secs
0,1.jpg,easyocr,Fluimucil 60Omg #15 Sig: Dissolve tablet in Yz...,16.155894
1,10.jpg,easyocr,Amoxicillin 500mg cap #21 Sig: take cap 3x a d...,7.606856
2,2.jpg,easyocr,Fluimucil 600mg Dissolve tablet in Yz glass wa...,8.450118
3,3.jpg,easyocr,Fluimucil 600mg Dissolve tablet in Yz glass wa...,7.372327
4,4.jpg,easyocr,Decolgen Neozep #15 Sig: Take tablet 3xlday fo...,8.490275
5,5.jpg,easyocr,Decolgen Neozep tablet 3xlday for 5 days\n#15\...,8.372041
6,6.jpg,easyocr,Decolgen Neozep tablet three times day for 5 d...,7.426643
7,7.jpg,easyocr,Betadine Gargle\nSig: Gargle 3xlday\nDifflam l...,8.471255
8,8.jpg,easyocr,Betadine Gargle Gargle 3xlday\nDifflam Lozenge...,8.721966
9,9.jpg,easyocr,Betadine Gargle Gargle three times day\nDiffla...,8.074740


In [23]:
for r in easyocr_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[1.jpg]
Fluimucil 60Omg #15 Sig: Dissolve tablet in Yz glass water once a day for 5 days
Immunosin 500mg #21 Sig: Take 1 tablet 3xlday for 7 days


[10.jpg]
Amoxicillin 500mg cap #21 Sig: take cap 3x a day X week


[2.jpg]
Fluimucil 600mg Dissolve tablet in Yz glass water Ixlday:
#15
Immunosin 50Omg 3xlday
#21


[3.jpg]
Fluimucil 600mg Dissolve tablet in Yz glass water once day:
#15
Immunosin 50Omg Three times day
#21


[4.jpg]
Decolgen Neozep #15 Sig: Take tablet 3xlday for 5 days
Loratadine 1Omg #21 Sig: Take tablet at breakfast for 14 days


[5.jpg]
Decolgen Neozep tablet 3xlday for 5 days
#15
Loratadine 1Omg tablet Ixlday at breakfast for 14 days
#21


[6.jpg]
Decolgen Neozep tablet three times day for 5 days
#15
Loratadine 1Omg tablet once day at breakfast for 14 days
#21


[7.jpg]
Betadine Gargle
Sig: Gargle 3xlday
Difflam lozenges #15 Sig: Every 4 hours


[8.jpg]
Betadine Gargle Gargle 3xlday
Difflam Lozenges Ixl4hours
#15


[9.jpg]
Betadine Gargle Gargle three times day
Difflam

# **3.0: PaddleOCR**

In [24]:
import paddleocr
from paddleocr import PaddleOCR

In [26]:
paddle_ocr = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False)

/usr/local/lib/python3.12/dist-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_rec`.


In [27]:
paddle_results = []

In [28]:
current_model = "paddleocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = paddle_ocr.predict(input=img)[0]['rec_texts']
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        paddle_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/1.jpg...
Processing: ocr-models/data/images/10.jpg...
Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...


In [29]:
pdocr_df = pd.DataFrame([r.__dict__ for r in paddle_results])
pdocr_df

,filename,model,pred_text,secs
0,1.jpg,paddleocr,"[1. Fluimucil 600mg, #15, Sig: Dissolve 1table...",10.468377
1,10.jpg,paddleocr,"[Amoxicillin 500mg cap #21, Sig: take 1 cap 3x...",8.161668
2,2.jpg,paddleocr,"[Fluimucil600mg, #15, Dissolve 1tablet in ½ gl...",11.604947
3,3.jpg,paddleocr,"[Fluimucil 600mg, #15, Dissolve 1 tablet in ½ ...",9.066459
4,4.jpg,paddleocr,"[1.Decolgen/Neozep, #15, Sig: Take 1 tablet 3x...",8.324847
5,5.jpg,paddleocr,"[Decolgen/Neozep, #15, 1 tablet 3x/day for 5 d...",7.547976
6,6.jpg,paddleocr,"[Decolgen/Neozep, #15, 1 tablet three times a ...",9.382298
7,7.jpg,paddleocr,"[1. Betadine Gargle, #1, Sig: Gargle 3x/day, 2...",5.850888
8,8.jpg,paddleocr,"[Betadine Gargle, #1, Gargle 3x/day, Diffl am ...",6.988640
9,9.jpg,paddleocr,"[Betadine Gargle, #1, Gargle three times a day...",6.339953


In [30]:
for r in paddle_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[1.jpg]
['1. Fluimucil 600mg', '#15', 'Sig: Dissolve 1tablet in ½ glass water once a', 'day for 5 days', '2. Immunosin 500mg', '#21', 'Sig: Take 1 tablet 3x/day for 7 days']


[10.jpg]
['Amoxicillin 500mg cap #21', 'Sig: take 1 cap 3x a day x 1 week']


[2.jpg]
['Fluimucil600mg', '#15', 'Dissolve 1tablet in ½ glass water 1x/day.', 'Immunosin 500mg', '#21', '3x/day']


[3.jpg]
['Fluimucil 600mg', '#15', 'Dissolve 1 tablet in ½ glass water once a day', 'Immunosin 500mg', '#21', 'Three times a day']


[4.jpg]
['1.Decolgen/Neozep', '#15', 'Sig: Take 1 tablet 3x/day for 5 days', '2.Loratadine 10mg', '#21', 'Sig: Take 1 tablet at breakfast for 14 days']


[5.jpg]
['Decolgen/Neozep', '#15', '1 tablet 3x/day for 5 days', 'Loratadine 10mg', '#21', '1 tablet 1x/day at breakfast for 14 days']


[6.jpg]
['Decolgen/Neozep', '#15', '1 tablet three times a day for 5 days', 'Loratadine 10mg', '#21', '1 tablet once a day at breakfast for 14 days']


[7.jpg]
['1. Betadine Gargle', '#1', 'Sig: Gargle 3x/

# **4.0: DocTR**

In [55]:
import doctr
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
from PIL import Image
import pathlib

In [40]:
doctr_predictor = ocr_predictor(pretrained=True)

In [81]:
def ocr_doctr(img_path: str) -> str:
    # DocTR expects a file or batch; we pass a single image
    doc = DocumentFile.from_images(img_path)
    res = doctr_predictor(doc)
    # extract text blocks in reading order
    lines = []
    for page in res.pages:
        for block in page.blocks:
            for line in block.lines:
                line_text = " ".join([w.value for w in line.words])
                if line_text.strip():
                    lines.append(line_text)
    return "\n".join(lines)

In [82]:
doctr_results = []

In [83]:
current_model = "doctr"

for img_path in images:
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_doctr(str(img_path))
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        doctr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/1.jpg...
Processing: ocr-models/data/images/10.jpg...
Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...


In [85]:
doctr_df = pd.DataFrame([r.__dict__ for r in doctr_results])
doctr_df

,filename,model,pred_text,secs
0,1.jpg,doctr,1. Fluimucil 600mg\n#15\nSig: Dissolve 1 table...,7.368129
1,10.jpg,doctr,Amoxicillin 500mg cap #21\nSig: take 1 cap 3x ...,5.053071
2,2.jpg,doctr,Fluimucil 600mg\n#15\nDissolve 1 tablet in % g...,6.287942
3,3.jpg,doctr,Fluimucil 600mg\n#15\nDissolve 1 tablet in % g...,5.423163
4,4.jpg,doctr,1. Decolgen/Neozep\n#15\nSig: Take 1 tablet 3x...,6.886792
5,5.jpg,doctr,Decolgen/Neozep\n#15\n1 tablet 3x/day for 5 da...,5.413042
6,6.jpg,doctr,Decolgen/Neozep\n#15\n1 tablet three times a d...,6.861810
7,7.jpg,doctr,1. Betadine Gargle\n#1\nSig: Gargle 3x/day\n2....,5.177352
8,8.jpg,doctr,Betadine Gargle\n#1\nGargle 3x/day\nDifflam Lo...,5.221790
9,9.jpg,doctr,Betadine Gargle\n#1\nGargle three times a day\...,6.023004


In [86]:
for r in doctr_results:
  print("[" + r.filename + "]")
  print(r.pred_text)
  print("\n")

[1.jpg]
1. Fluimucil 600mg
#15
Sig: Dissolve 1 tablet in % glass water once a
day for 5 days
2. Immunosin 500mg
#21
Sig: Take 1 tablet 3x/day for 7 days


[10.jpg]
Amoxicillin 500mg cap #21
Sig: take 1 cap 3x a day X 1 week


[2.jpg]
Fluimucil 600mg
#15
Dissolve 1 tablet in % glass water 1X/day.
Immunosin 500mg
#21
3x/day


[3.jpg]
Fluimucil 600mg
#15
Dissolve 1 tablet in % glass water once: a day.
Immunosin 500mg
#21
Three times a day


[4.jpg]
1. Decolgen/Neozep
#15
Sig: Take 1 tablet 3x/day for 5 days
2. Loratadine 10mg
#21
Sig: Take 1 tablet at breakfast for 14 days


[5.jpg]
Decolgen/Neozep
#15
1 tablet 3x/day for 5 days
Loratadine 10mg
#21
1 tablet 1x/day at breakfast for 14 days


[6.jpg]
Decolgen/Neozep
#15
1 tablet three times a day for 5 days
Loratadine 10mg
#21
1 tablet once a day at breaktast for 14 days


[7.jpg]
1. Betadine Gargle
#1
Sig: Gargle 3x/day
2. Difflam lozenges
#15
Sig: Every 4 hours


[8.jpg]
Betadine Gargle
#1
Gargle 3x/day
Difflam Lozenges
#15
1x/4hours


[9

# **5.0: TrOCR**

In [87]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [88]:
trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [106]:
def show_img(path: str):
  img = Image.open(path).convert("RGB")
  return img

In [99]:
def ocr(path):
    pixel_values = trocr_processor(images=path, return_tensors="pt").pixel_values
    outputs = trocr_model.generate(pixel_values)
    return trocr_processor.batch_decode(outputs, skip_special_tokens=True)[0]

In [102]:
ocr(show_img("ocr-models/data/images/1.jpg"))

'THANK'

In [90]:
def ocr_trocr(img_rgb: np.ndarray) -> str:
    pixel_values = trocr_processor(images=Image.fromarray(img_rgb), return_tensors="pt").pixel_values
    outputs = trocr_model.generate(pixel_values)
    return trocr_processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

In [91]:
trocr_results = []

In [92]:
current_model = "trocr"

for img_path in images:
        img = load_image(img_path)
        start = time.time()
        print("Processing: " + str(img_path) + "...")
        try:
            pred = ocr_trocr(img)
        except Exception as e:
            pred = f"[ERROR] {e}"
        secs = time.time() - start
        trocr_results.append(OCRResult(filename=img_path.name, model=current_model, pred_text=pred, secs=secs))

Processing: ocr-models/data/images/1.jpg...
Processing: ocr-models/data/images/10.jpg...
Processing: ocr-models/data/images/2.jpg...
Processing: ocr-models/data/images/3.jpg...
Processing: ocr-models/data/images/4.jpg...
Processing: ocr-models/data/images/5.jpg...
Processing: ocr-models/data/images/6.jpg...
Processing: ocr-models/data/images/7.jpg...
Processing: ocr-models/data/images/8.jpg...
Processing: ocr-models/data/images/9.jpg...


In [93]:
trocr_df = pd.DataFrame([r.__dict__ for r in trocr_results])
trocr_df

,filename,model,pred_text,secs
0,1.jpg,trocr,THANK,28.533364
1,10.jpg,trocr,................,16.256015
2,2.jpg,trocr,CASHIERIER,32.453946
3,3.jpg,trocr,***,15.460504
4,4.jpg,trocr,CASHIER,29.213832
5,5.jpg,trocr,................,15.264077
6,6.jpg,trocr,:,15.582209
7,7.jpg,trocr,***,14.817426
8,8.jpg,trocr,:,14.885719
9,9.jpg,trocr,:,14.952267


# **6.0: Keras-OCR**

In [109]:
import matplotlib.pyplot as plt
import keras_ocr

AttributeError: `np.sctypes` was removed in the NumPy 2.0 release. Access dtypes explicitly instead.